In [ ]:
#관측자 "관측 우선도(U)를 고정했을 때, 양자 시스템의 출력 결과(φ)가 환경 조건(E_i)에 따라 얼마나 일관된 구조를 유지하는지 확인한다."
#이 실험은 관측자 함수적 개입이 회로 전체에 구조적 구속력을 행사하는가를 검증합니다.
#시뮬레이터 설정: qasm_simulator 사용
#jupyter lab

import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
from scipy.spatial.distance import cosine

def build_circuit(U, noise=False):
    n = len(U)
    qc = QuantumCircuit(n, n)

    # 관측 우선도에 따른 Ry 게이트
    for i, u in enumerate(U):
        theta = u * np.pi
        qc.ry(theta, i)

    # 환경 조건 (노이즈 삽입)
    if noise:
        for i in range(n):
            if np.random.rand() < 0.3:  # 30% 확률로 X 노이즈
                qc.x(i)

    qc.barrier()
    qc.measure(range(n), range(n))
    return qc

# 시뮬레이터 설정
sim = Aer.get_backend('qasm_simulator')

# 관측 우선도 설정
U = [0.2, 0.7, 0.9]

# 기준 회로
qc_ref = build_circuit(U, noise=False)
res_ref = execute(qc_ref, sim, shots=1024).result()
counts_ref = res_ref.get_counts()

# 환경 조건 변화 실험
similarities = []
for _ in range(10):
    qc_noisy = build_circuit(U, noise=True)
    res = execute(qc_noisy, sim, shots=1024).result()
    counts = res.get_counts()

    # 두 분포 유사도 비교
    keys = set(counts_ref.keys()).union(counts.keys())
    v1 = np.array([counts_ref.get(k, 0) for k in keys])
    v2 = np.array([counts.get(k, 0) for k in keys])
    sim_score = 1 - cosine(v1, v2)
    similarities.append(sim_score)

# 출력
print("관측 우선도 고정 상태에서 환경 변화에 따른 유사도 평균:", np.mean(similarities))



# result 관측 우선도 고정 상태에서 환경 변화에 따른 유사도 평균: 0.6207259763218352

In [ ]:
# “관측 우선도(U)는 의도를 표현한 것이다. 관측은 상태를 정하고, 정해진 상태는 회로 내에서 구조적 확장 또는 연결을 가능하게 한다. 그렇게 정해진 결과는 다시 관측자의 우선도와의 일치성을 강화하고, 이는 관측을 가속화한다.”
# 출력 분포의 유사도 증가 또는 엔트로피 감소로 구조 수렴 여부 평가
#시뮬레이터 설정: qasm_simulator 사용
#jupyter lab

import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from scipy.spatial.distance import cosine
from scipy.stats import entropy

# 회로 생성 함수
def build_circuit(U):
    n = len(U)
    qc = QuantumCircuit(n, n)
    for i, u in enumerate(U):
        qc.ry(u * np.pi, i)
    qc.barrier()
    qc.measure(range(n), range(n))
    return qc

# 우선도 갱신 함수 (출력 빈도를 우선도에 반영)
def update_priority(counts, n):
    total = sum(counts.values())
    state_probs = np.zeros(n)
    
    for bitstring, freq in counts.items():
        prob = freq / total
        bits = [int(b) for b in bitstring[::-1]]  # Qiskit 비트 순서 보정
        state_probs += np.array(bits) * prob
    
    # 정규화: 0~1 사이로 유지
    return np.clip(state_probs, 0.01, 0.99)

# 출력 분포를 벡터로 변환
def counts_to_vector(counts, keys):
    return np.array([counts.get(k, 0) for k in keys], dtype=np.float64)

# 엔트로피 계산
def counts_entropy(counts):
    total = sum(counts.values())
    probs = np.array([v / total for v in counts.values() if v > 0])
    return entropy(probs, base=2)

# 시뮬레이터
sim = Aer.get_backend('qasm_simulator')

# 초기 우선도 (의도)
U = np.array([0.2, 0.7, 0.9])
n = len(U)

# 기록용
all_U = [U.copy()]
all_entropies = []
all_similarities = []

# 초기 기준 회로 실행
qc0 = build_circuit(U)
res0 = execute(qc0, sim, shots=1024).result()
counts0 = res0.get_counts()
keys_all = set(counts0.keys())

all_entropies.append(counts_entropy(counts0))

# 피드백 루프
for t in range(1, 11):
    qc = build_circuit(U)
    res = execute(qc, sim, shots=1024).result()
    counts = res.get_counts()
    keys_all |= set(counts.keys())

    # 구조 유사도
    v0 = counts_to_vector(counts0, keys_all)
    vt = counts_to_vector(counts, keys_all)
    sim_score = 1 - cosine(v0, vt)
    all_similarities.append(sim_score)

    # 엔트로피
    ent = counts_entropy(counts)
    all_entropies.append(ent)

    # 다음 우선도 갱신
    U = update_priority(counts, n)
    all_U.append(U.copy())

# 결과 출력
print("\n=== 반복적 피드백 실험 결과 ===")
for t in range(11):
    print(f"{t:2d}단계 | 우선도: {np.round(all_U[t], 3)} | 엔트로피: {all_entropies[t]:.3f}" +
          (f" | 유사도: {all_similarities[t-1]:.3f}" if t > 0 else ""))



#result
#=== 반복적 피드백 실험 결과 ===
 #0단계 | 우선도: [0.2 0.7 0.9] | 엔트로피: 1.357
 #1단계 | 우선도: [0.084 0.788 0.981] | 엔트로피: 1.292 | 유사도: 1.000
 #2단계 | 우선도: [0.016 0.895 0.99 ] | 엔트로피: 0.613 | 유사도: 0.982
 #3단계 | 우선도: [0.01  0.981 0.99 ] | 엔트로피: 0.144 | 유사도: 0.960
 #4단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.011 | 유사도: 0.955
 #5단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.022 | 유사도: 0.955
 #6단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.011 | 유사도: 0.955
 #7단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.011 | 유사도: 0.955
 #8단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.000 | 유사도: 0.955
 #9단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.011 | 유사도: 0.955
 #10단계 | 우선도: [0.01 0.99 0.99] | 엔트로피: 0.000 | 유사도: 0.955